## DeepLearning Model

#### Import libraries

In [4]:
import joblib
import pandas as pd
import sklearn
#import sklearn.datasets
from keras.models import Sequential
from keras.layers import Dense
import warnings

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)

tr_features = pd.read_csv('../Resources/train_features.csv')
tr_labels = pd.read_csv('../Resources/train_labels.csv', header=None)
test_features = pd.read_csv('../Resources/test_features.csv')
test_labels = pd.read_csv('../Resources/test_labels.csv', header=None)

val_features = pd.read_csv('../Resources/val_features.csv')
val_labels = pd.read_csv('../Resources/val_labels.csv', header=None)

In [3]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))

    means = results.cv_results_['mean_test_score']
    stds = results.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, results.cv_results_['params']):
        print('{} (+/-{}) for {}'.format(round(mean, 3), round(std * 2, 3), params))

In [5]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(tr_features)
X_train_scaled = X_scaler.transform(tr_features)
X_test_scaled = X_scaler.transform(test_features)
X_val_scaled = X_scaler.transform(val_features)

#### Train Model

In [6]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=100, activation='relu', input_dim=40))
model.add(Dense(units=3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
#sparse_categorical_crossentropy was used because of multiple levels of the y-value (0, 1, 2)
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               4100      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(
X_train_scaled,
tr_labels.values,
epochs=60,
shuffle=True,
verbose=2
)

Epoch 1/60
 - 1s - loss: 0.6226 - accuracy: 0.6800
Epoch 2/60
 - 0s - loss: 0.3684 - accuracy: 0.8107
Epoch 3/60
 - 0s - loss: 0.3528 - accuracy: 0.8135
Epoch 4/60
 - 0s - loss: 0.3502 - accuracy: 0.8331
Epoch 5/60
 - 0s - loss: 0.3345 - accuracy: 0.8336
Epoch 6/60
 - 0s - loss: 0.3292 - accuracy: 0.8331
Epoch 7/60
 - 0s - loss: 0.3220 - accuracy: 0.8448
Epoch 8/60
 - 0s - loss: 0.3178 - accuracy: 0.8484
Epoch 9/60
 - 0s - loss: 0.3079 - accuracy: 0.8569
Epoch 10/60
 - 0s - loss: 0.3111 - accuracy: 0.8617
Epoch 11/60
 - 0s - loss: 0.3118 - accuracy: 0.8510
Epoch 12/60
 - 0s - loss: 0.3162 - accuracy: 0.8498
Epoch 13/60
 - 0s - loss: 0.3032 - accuracy: 0.8636
Epoch 14/60
 - 0s - loss: 0.3004 - accuracy: 0.8643
Epoch 15/60
 - 0s - loss: 0.2969 - accuracy: 0.8660
Epoch 16/60
 - 0s - loss: 0.2999 - accuracy: 0.8646
Epoch 17/60
 - 0s - loss: 0.2903 - accuracy: 0.8696
Epoch 18/60
 - 0s - loss: 0.2903 - accuracy: 0.8689
Epoch 19/60
 - 0s - loss: 0.2880 - accuracy: 0.8686
Epoch 20/60
 - 0s - l

#### Model evaluation and Prediction on test-data

In [9]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, test_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.25757277811783563, Accuracy: 0.8956397175788879


In [10]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
Actual_label = (test_labels[:20]).values.ravel()

In [11]:
result_df = pd.DataFrame({
    'Prediction': encoded_predictions,
    'Actual': Actual_label
})
result_df

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,0,0
4,0,1
5,2,2
6,1,1
7,0,0
8,2,2
9,2,2


#### Write Model

In [14]:
joblib.dump(model, '../Saved_model/DL_model.pkl')

['../Saved_model/DL_model.pkl']

#### Model validation using validation-data

In [15]:
 model_val = joblib.load('../Saved_model/DL_model.pkl')

In [16]:
model_loss, model_accuracy = model_val.evaluate(
    X_val_scaled, val_labels, verbose=2)
print(
    f"result_df Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

result_df Neural Network - Loss: 0.27126866365877517, Accuracy: 0.8905579447746277
